# Part I (Analytical Option Formulae)
#### Consider the following European options:
- Vanilla call/put
- Digital cash-or-nothing call/put
- Digital asset-or-nothing call/put

#### Derive and implement the following models to value these options in Python:
1. Black-Scholes model
2. Bachelier model
3. Black model
4. Displaced-diffusion model

# Response
##### What is a **vanilla** option?
A **vanilla** option is a financial instrument that gives the holder the right, but not the obligation, to buy or sell an underlying asset at a predetermined price within a given timeframe. This option is a call/put option that has no special nor unusual features, meaning it is standardised if traded on an exchange, e.g., SGX.
##### What is a **cash/asset-or-nothing** call/put?
A **cash/asset-or-nothing** call (resp. put) is an option that has a binary outcome: it pays out either a fixed amount of one unit of cash/asset, if the underlying stock becomes above (resp. below) a predetermined threshold or strike price, or pays out nothing.
##### The standard normal distribution
Let $Z \sim N(0,1)$ be the standard normal random variable, whose probability density function (PDF) is given by $\phi(z) = \frac{1}{\sqrt{2\pi}}e^{-\frac{z^2}{2}}$. Define its cumulative distribution function (CDF) by $\Phi(z)=\int_{-\infty}^z\phi(z')\,dz'$.
##### Define the following variables:
$t$: the time in years, where $t\geq0$; assuming WLOG that $t=0$ represents the current year,\
$r$: the annual continuously compounded risk-free interest rate,\
$S_0$: the initial asset/stock price,\
$\mu$: the annual drift rate of $S_0$,\
$\sigma$: the standard deviation of the stock's returns, i.e., the square root of the quadratic variation of the stock's log price process (measure of stock's volatility),\
$T$: the expiry date of the option, i.e., $0\leq t\leq T$,\
$K$: the ATM (at the money) strike price of the option.

We start by importing the following modules needed for our computation:

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import norm

As always, we first recall Itô's lemma: Let $X_t=f(t,S_t)$. Then, $dX_t=\frac{\partial f}{\partial t}\,dt+\frac{\partial f}{\partial S_t}\,dS_t+\frac{1}{2}\frac{\partial^2f}{\partial S_t^2}(dS_t)^2$.

## 1. Black-Scholes model
The prices of the European vanilla options at expiry date $T$ are given by the following formulae:
- Call: $C_{BS,v} = S_0\Phi(d_1) - Ke^{-rT}\Phi(d_2)$, where $d_1 = \frac{\log\frac{S_0}{K} + \left(r + \frac{\sigma^2}{2}\right)T}{\sigma\sqrt{T}}$, $d_2 = d_1 - \sigma\sqrt{T}$,
- Put: $P_{BS,v} = Ke^{-rT}\Phi(-d_2) - S_0\Phi(-d_1)$.

**Derivation**<br>
We start from the Black-Scholes PDE for the stock price process:
  \begin{equation*}
    \begin{split}
      dS_t&=rS_t\,dt+\sigma S_t\,dW_t\\
      d(\log S_t)&=\frac{dS_t}{S_t}-\frac{(dS_t)^2}{2S_t^2}\,\,\,\,[\text{by Itô's lemma}]\\
      \log\frac{S_t}{S_0}=\log S_t-\log S_0&=\left(r-\frac{\sigma^2}{2}\right)t+\sigma W_t\,\,\,\,[\text{using box calculus with}\,\,(dW_t)^2=dt]\\
      S_t&=S_0\exp\left[\left(r-\frac{\sigma^2}{2}\right)t+\sigma W_t\right],
    \end{split}
  \end{equation*}
where $W_t\sim N(0,t)$ is the Brownian motion with variance $t$. For simplicity, let $x=\frac{W_t}{\sqrt{t}}\sim N(0,1)$ be the standard normal random variable. Assuming a payoff of $\max\left\{S_T-K,0\right\}$, we have, for the case $S_T>K$:
  \begin{equation*}
    \begin{split}
      \log\frac{S_T}{K}=\log\frac{S_0}{K}+\left(r-\frac{\sigma^2}{2}\right)T+\sigma\sqrt{T}x&>0\\
      x&>\frac{\log\frac{K}{S_0}-\left(r-\frac{\sigma^2}{2}\right)T}{\sigma\sqrt{T}}=x^*.
    \end{split}
  \end{equation*}
The option pricing formula corresponding to the call option can then be derived as follows:
  \begin{equation*}
    \begin{split}
      C_{BS,v}&=e^{-rT}\mathbb{E}\left[\max\left\{S_T-K,0\right\}\right]\\
      &=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty\left[S_0\exp\left[\left(r-\frac{\sigma^2}{2}\right)T+\sigma\sqrt{T}x\right]-K\right]e^{-\frac{x^2}{2}}\,dx\\
      &=\frac{S_0}{\sqrt{2\pi}}\int_{x^*}^\infty\exp\left[-\frac{\sigma^2}{2}T+\sigma\sqrt{T}x-\frac{x^2}{2}\right]\,dx-\frac{Ke^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty e^{-\frac{x^2}{2}}\,dx\\
      &=\frac{S_0}{\sqrt{2\pi}}\int_{x^*}^\infty e^{-\frac{(x-\sigma\sqrt{T})^2}{2}}\,dx-\frac{Ke^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty e^{-\frac{x^2}{2}}\,dx\\
      &=S_0[\Phi(\infty)-\Phi(x^*-\sigma\sqrt{T})]-Ke^{-rT}[\Phi(\infty)-\Phi(x^*)]\\
      &=S_0\Phi(-x^*+\sigma\sqrt{T})-Ke^{-rT}\Phi(-x^*)\\
      &=S_0\Phi(d_1)-Ke^{-rT}\Phi(d_2),
    \end{split}
  \end{equation*}
where $d_1=-x^*+\sigma\sqrt{T}=\frac{\log\frac{S_0}{K}+\left(r-\frac{\sigma^2}{2}\right)T}{\sigma\sqrt{T}}+\sigma\sqrt{T}=\frac{\log\frac{S_0}{K}+\left(r+\frac{\sigma^2}{2}\right)T}{\sigma\sqrt{T}}$ and $d_2=d_1-\sigma\sqrt{T}$.

Similarly, the option pricing formula corresponding to the put $(S_T<K)$ option can be derived as follows:
  \begin{equation*}
    \begin{split}
      P_{BS,v}&=e^{-rT}\mathbb{E}\left[\max\left\{K-S_T,0\right\}\right]\\
      &=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{x^*}\left[K-S_0\exp\left[\left(r-\frac{\sigma^2}{2}\right)T+\sigma\sqrt{T}x\right]\right]e^{-\frac{x^2}{2}}\,dx\\
      &=\frac{Ke^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{x^*}e^{-\frac{x^2}{2}}\,dx-\frac{S_0}{\sqrt{2\pi}}\int_{-\infty}^{x^*}e^{-\frac{(x-\sigma\sqrt{T})^2}{2}}\,dx\\
      &=Ke^{-rT}\Phi(x^*)-S_0\Phi(x^*-\sigma\sqrt{T})\\
      &=Ke^{-rT}\Phi(-d_2)-S_0\Phi(-d_1).
    \end{split}
  \end{equation*}

In [5]:
# In Python, we define these functions below:
def C_BSv(S0, K, r, sigma, T):
    d1 = (np.log(S0/K) + (r + sigma**2/2) * T)/(sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
def P_BSv(S0, K, r, sigma, T):
    d1 = (np.log(S0/K) + (r + sigma**2/2) * T)/(sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)

The prices of the European cash-or-nothing options at expiry date $T$ are given by the following formulae:
- Call: $C_{BS,c} = e^{-rT}\Phi(d_2)$,
- Put: $P_{BS,c} = e^{-rT}\Phi(-d_2)$.

**Derivation**<br>
Once again, we start from the Black-Scholes PDE. Assuming a payoff of $\mathbb{1}_{S_T>K}$ corresponding to the call option, the option pricing formula corresponding to the call option can be derived as follows: $C_{BS,c}=e^{-rT}\mathbb{E}\left[\mathbb{1}_{S_T>K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty e^{-\frac{x^2}{2}}\,dx=e^{-rT}\Phi(d_2)$.

Similarly, the option pricing formula corresponding to the put option is: $P_{BS,c}=e^{-rT}\mathbb{E}\left[\mathbb{1}_{S_T<K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{x^*}e^{-\frac{x^2}{2}}\,dx=e^{-rT}\Phi(-d_2)$.

In [7]:
# In Python, we define these functions below:
def C_BSc(S0, K, r, sigma, T):
    d2 = (np.log(S0/K) + (r + sigma**2/2) * T)/(sigma * np.sqrt(T)) - sigma * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(d2)
def P_BSc(S0, K, r, sigma, T):
    d2 = (np.log(S0/K) + (r + sigma**2/2) * T)/(sigma * np.sqrt(T)) - sigma * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(-d2)

The prices of the European asset-or-nothing options at expiry date $T$ are given by the following formulae:
- Call: $C_{BS,a} = S_0\Phi(d_1)$,
- Put: $P_{BS,a} = S_0\Phi(-d_1)$.

**Derivation**<br>
Once again, we start from the Black-Scholes PDE. Assuming a payoff of $S_T\mathbb{1}_{S_T>K}$ corresponding to the call option, the option pricing formula corresponding to the call option can be derived as follows: $C_{BS,a}=e^{-rT}\mathbb{E}\left[S_T\mathbb{1}_{S_T>K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty S_Te^{-\frac{x^2}{2}}\,dx=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty S_0\exp\left[\left(r-\frac{\sigma^2}{2}\right)t+\sigma\sqrt{T}x\right]e^{-\frac{x^2}{2}}\,dx=S_0\Phi(d_1)$.

Similarly, the option pricing formula corresponding to the put option is: $P_{BS,a}=e^{-rT}\mathbb{E}\left[S_T\mathbb{1}_{S_T<K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{x^*}S_Te^{-\frac{x^2}{2}}\,dx=S_0\Phi(-d_1)$.

In [9]:
# In Python, we define these functions below:
def C_BSa(S0, K, r, sigma, T):
    d1 = (np.log(S0/K) + (r + sigma**2/2) * T)/(sigma * np.sqrt(T))
    return S0 * norm.cdf(d1)
def P_BSa(S0, K, r, sigma, T):
    d1 = (np.log(S0/K) + (r + sigma**2/2) * T)/(sigma * np.sqrt(T))
    return S0 * norm.cdf(-d1)

## 2. Bachelier model
The prices of the European vanilla options at expiry date $T$ are given by the following formulae:
- Call: $C_{Ba,v} = e^{-rT}\left[(S_0 - K)\Phi(d) + \sigma\sqrt{T}\phi(d)\right]$, where $d = \frac{S_0 - K}{\sigma\sqrt{T}}$,
- Put: $P_{Ba,v} = e^{-rT}\left[(K - S_0)\Phi(-d) + \sigma\sqrt{T}\phi(-d)\right]$.

**Derivation**<br>
We start from the Bachelier PDE for the stock price process: $dS_t=\sigma\,dW_t\Rightarrow S_t=S_0+\sigma W_t=S_0+\sigma\sqrt{t}x$. Assuming a payoff of $\max\left\{S_T-K,0\right\}$, we have, for the case $S_T>K$: $S_0-K+\sigma\sqrt{T}x>0\Rightarrow x>\frac{K-S_0}{\sigma\sqrt{T}}=x^*$.
The option pricing formula corresponding to the call option can then be derived as follows:
  \begin{equation*}
    \begin{split}
      C_{Ba,v}&=e^{-rT}\mathbb{E}\left[\max\left\{S_T-K,0\right\}\right]\\
      &=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty\left(S_0-K+\sigma\sqrt{T}x\right)e^{-\frac{x^2}{2}}\,dx\\
      &=e^{-rT}\left[(S_0-K)\Phi(d)+\sigma\sqrt{T}\phi(d)\right],\,\text{where}\,\,d=-x^*=\frac{S_0-K}{\sigma\sqrt{T}}.
    \end{split}
  \end{equation*}

Similarly, the option pricing formula corresponding to the put option can be derived as follows:
  \begin{equation*}
    \begin{split}
      P_{Ba,v}&=e^{-rT}\mathbb{E}\left[\max\left\{K-S_T,0\right\}\right]\\
      &=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{x^*}\left(K-S_0-\sigma\sqrt{T}x\right)e^{-\frac{x^2}{2}}\,dx\\
      &=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{x^*}(K-S_0)e^{-\frac{x^2}{2}}\,dx-\frac{\sigma\sqrt{T}e^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{x^*}xe^{-\frac{x^2}{2}}\,dx\\
      &=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{-x^*}^{\infty}(K-S_0)e^{-\frac{x^2}{2}}\,dx+\frac{\sigma\sqrt{T}e^{-rT}}{\sqrt{2\pi}}\int_{-x^*}^{\infty}xe^{-\frac{x^2}{2}}\,dx\\
      &=e^{-rT}\left[(K-S_0)\Phi(-d)+\sigma\sqrt{T}\phi(-d)\right].
    \end{split}
  \end{equation*}

In [11]:
# In Python, we define these functions below:
def C_Bav(S0, K, r, sigma, T):
    d = (S0 - K)/(sigma * np.sqrt(T))
    return np.exp(-r * T) * ((S0 - K) * norm.cdf(d) + sigma * np.sqrt(T) * norm.pdf(d))
def P_Bav(S0, K, r, sigma, T):
    d = (S0 - K)/(sigma * np.sqrt(T))
    return np.exp(-r * T) * ((K - S0) * norm.cdf(-d) + sigma * np.sqrt(T) * norm.pdf(-d))

The prices of the European cash-or-nothing options at expiry date $T$ are given by the following formulae:
- Call: $C_{Ba,c} = e^{-rT}\Phi(d)$,
- Put: $P_{Ba,c} = e^{-rT}\Phi(-d)$.

**Derivation**<br>
Once again, we start from the Bachelier PDE. Assuming a payoff of $\mathbb{1}_{S_T>K}$ corresponding to the call option, the option pricing formula corresponding to the call option can be derived as follows: $C_{Ba,c}=e^{-rT}\mathbb{E}\left[\mathbb{1}_{S_T>K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty e^{-\frac{x^2}{2}}\,dx=e^{-rT}\Phi(d)$.

Similarly, the option pricing formula corresponding to the put option is: $P_{Ba,c}=e^{-rT}\mathbb{E}\left[\mathbb{1}_{S_T<K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{x^*}e^{-\frac{x^2}{2}}\,dx=e^{-rT}\Phi(-d)$.

In [13]:
# In Python, we define these functions below:
def C_Bac(S0, K, r, sigma, T):
    d = (S0 - K)/(sigma * np.sqrt(T))
    return np.exp(-r * T) * norm.cdf(d)
def P_Bac(S0, K, r, sigma, T):
    d = (S0 - K)/(sigma * np.sqrt(T))
    return np.exp(-r * T) * norm.cdf(-d)

The prices of the European asset-or-nothing options at expiry date $T$ are given by the following formulae:
- Call: $C_{Ba,a} = S_0\Phi(d) + \sigma\sqrt{T}\phi(d)$,
- Put: $P_{Ba,a} = S_0\Phi(-d) + \sigma\sqrt{T}\phi(-d)$.

**Derivation**<br>
Once again, we start from the Bachelier PDE. Assuming a payoff of $S_T\mathbb{1}_{S_T>K}$ corresponding to the call option, the option pricing formula corresponding to the call option can be derived as follows: $C_{Ba,a}=e^{-rT}\mathbb{E}\left[S_T\mathbb{1}_{S_T>K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty S_Te^{-\frac{x^2}{2}}\,dx=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty(S_0+\sigma\sqrt{T}x)e^{-\frac{x^2}{2}}\,dx=S_0\Phi(d)+\sigma\sqrt{T}\phi(d)$.

Similarly, the option pricing formula corresponding to the put option is: $P_{Ba,a}=e^{-rT}\mathbb{E}\left[S_T\mathbb{1}_{S_T<K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{x^*}S_Te^{-\frac{x^2}{2}}\,dx=S_0\Phi(-d)+\sigma\sqrt{T}\phi(-d)$.

In [15]:
# In Python, we define these functions below:
def C_Baa(S0, K, r, sigma, T):
    d = (S0 - K)/(sigma * np.sqrt(T))
    return S0 * norm.cdf(d) + sigma * np.sqrt(T) * norm.pdf(d)
def P_Baa(S0, K, r, sigma, T):
    d = (S0 - K)/(sigma * np.sqrt(T))
    return S0 * norm.cdf(-d) + sigma * np.sqrt(T) * norm.pdf(-d)

## 3. Black model
Let $F_0 = S_0e^{rT}$ be the initial discounted futures price. The prices of the European vanilla options at expiry date $T$ are given by the following formulae:
- Call: $C_{B,v} = e^{-rT}\left[F_0\Phi(d_1) - K\Phi(d_2)\right]$, where $d_1 = \frac{\log\frac{F_0}{K} + \frac{\sigma^2}{2}T}{\sigma\sqrt{T}}$, $d_2 = d_1 - \sigma\sqrt{T}$,
- Put: $P_{B,v} = e^{-rT}\left[K\Phi(-d_2) - F_0\Phi(-d_1)\right]$.

**Derivation**<br>
The derivation is the same as the one for the Black-Scholes model using the Black-Scholes PDE for the stock price process, except that we replace $S_0$ with $F_0e^{-rT}$.

In [17]:
# In Python, we define these functions below:
def C_Bv(F0, K, r, sigma, T):
    d1 = (np.log(F0/K) + sigma**2/2 * T)/(sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) * (F0 * norm.cdf(d1) - K * norm.cdf(d2))
def P_Bv(F0, K, r, sigma, T):
    d1 = (np.log(F0/K) + sigma**2/2 * T)/(sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return np.exp(-r * T) * (K * norm.cdf(-d2) - F0 * norm.cdf(-d1))

The prices of the European cash-or-nothing options at expiry date $T$ are given by the following formulae:
- Call: $C_{B,c} = e^{-rT}\Phi(d_2)$,
- Put: $P_{B,c} = e^{-rT}\Phi(-d_2)$.

**Derivation**<br>
The derivation is the same as the one for the Black-Scholes model.

In [19]:
# In Python, we define these functions below:
def C_Bc(F0, K, r, sigma, T):
    d2 = (np.log(F0/K) + sigma**2/2 * T)/(sigma * np.sqrt(T)) - sigma * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(d2)
def P_Bc(F0, K, r, sigma, T):
    d2 = (np.log(F0/K) + sigma**2/2 * T)/(sigma * np.sqrt(T)) - sigma * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(-d2)

The prices of the European asset-or-nothing options at expiry date $T$ are given by the following formulae:
- Call: $C_{B,a} = F_0e^{-rT}\Phi(d_1)$,
- Put: $P_{B,a} = F_0e^{-rT}\Phi(-d_1)$.

**Derivation**<br>
The derivation is the same as the one for the Black-Scholes model using the Black-Scholes PDE for the stock price process, except that we replace $S_0$ with $F_0e^{-rT}$.

In [21]:
# In Python, we define these functions below:
def C_Ba(F0, K, r, sigma, T):
    d1 = (np.log(F0/K) + sigma**2/2 * T)/(sigma * np.sqrt(T))
    return F0 * np.exp(-r * T) * norm.cdf(d1)
def P_Ba(F0, K, r, sigma, t):
    d1 = (np.log(F0/K) + sigma**2/2 * T)/(sigma * np.sqrt(T))
    return F0 * np.exp(-r * T) * norm.cdf(-d1)

## 4. Displaced-diffusion model
In the displaced-diffusion model, the factor $\beta\in[0,1]$ plays a significant role in modifying the underlying asset's/stock's price dynamics:
- $\beta = 1$: The model reduces to the standard Black-Scholes model, where the volatility term is simply $\sigma S_0$, meaning the volatility is proportional to the asset's/stock's price.
- $\beta = 0$: The model's dynamics are dominated by the constant $F_0$, making the volatility independent of the asset's/stock's price. This implies the volatility term becomes $\sigma F_0$.
- $0 < \beta < 1$: The model incorporates a blend of both the asset's/stock's price $S_0$ and the constant $F_0$. This allows the model to capture scenarios where the volatility is not entirely proportional to the asset's/stock's price but is also influenced by some baseline, represented by $F_0$.

In essence, $\beta$ controls the degree to which the asset's/stock's price $S_0$ influences the volatility of the asset. A higher $\beta$ implies the volatility is more strongly tied to the asset's/stock's price, while a lower $\beta$ suggests the volatility is more dependent on the constant $F_0$. This flexibility makes the displaced-diffusion model useful in modelling situations where the asset's/stock's price volatility does not behave purely as a linear function of its current price.

The prices of the European vanilla options at expiry date $T$ are given by the following formulae:
- Call: $C_{D,v} = C_{B,v}\left(\frac{F_0}{\beta}, K + \frac{1 - \beta}{\beta}F_0, r, \beta\sigma, T\right) = e^{-rT}\left[\frac{F_0}{\beta}\Phi(d_1) - \left(K+\frac{1-\beta}{\beta}F_0\right)\Phi(d_2)\right]$, where $d_1 = \frac{\log\left(\frac{F_0}{F_0 + \beta(K-F_0)}\right) + \frac{\left(\sigma\beta\right)^2}{2}T}{\sigma\beta\sqrt{T}}$, $d_2 = d_1 - \sigma\beta\sqrt{T}$,
- Put: $P_{D,v} = P_{B,v}\left(\frac{F_0}{\beta}, K + \frac{1 - \beta}{\beta}F_0, r, \beta\sigma, T\right) = e^{-rT}\left[\left(K+\frac{1-\beta}{\beta}F_0\right)\Phi(-d_2) - \frac{F_0}{\beta}\Phi(-d_1))\right]$.

In [23]:
# In Python, we define these functions below:
def C_Dv(F0, K, r, sigma, T, beta):
    return C_Bv(F0/beta, K + ((1 - beta)/beta) * F0, r, beta * sigma, T)
def P_Dv(F0, K, r, sigma, T, beta):
    return P_Bv(F0/beta, K + ((1 - beta)/beta) * F0, r, beta * sigma, T)

The prices of the European cash-or-nothing options at expiry date $T$ are given by the following formulae:
- Call: $C_{D,c} = C_{B,c}\left(\frac{F_0}{\beta}, K + \frac{1 - \beta}{\beta}F_0, r, \beta\sigma, T\right) = e^{-rT}\Phi(d_2)$,
- Put: $P_{D,c} = P_{B,c}\left(\frac{F_0}{\beta}, K + \frac{1 - \beta}{\beta}F_0, r, \beta\sigma, T\right) = e^{-rT}\Phi(-d_2)$.

In [25]:
# In Python, we define these functions below:
def C_Dc(F0, K, r, sigma, T, beta):
    return C_Bc(F0/beta, K + ((1 - beta)/beta) * F0, r, beta * sigma, T)
def P_Dc(F0, K, r, sigma, T, beta):
    return P_Bc(F0/beta, K + ((1 - beta)/beta) * F0, r, beta * sigma, T)

The prices of the European asset-or-nothing options at expiry date $T$ are given by the following formulae:
- Call: $C_{D,a} = C_{B,a}\left(\frac{F_0}{\beta}, K + \frac{1 - \beta}{\beta}F_0, r, \beta\sigma, T\right) = \frac{F_0}{\beta}\Phi(d_1)$,
- Put: $P_{D,a} = P_{B,a}\left(\frac{F_0}{\beta}, K + \frac{1 - \beta}{\beta}F_0, r, \beta\sigma, T\right) = \frac{F_0}{\beta}\Phi(-d_1)$.

In [27]:
# In Python, we define these functions below:
def C_Da(F0, K, r, sigma, T, beta):
    return C_Ba(F0/beta, K + ((1 - beta)/beta) * F0, r, beta * sigma, T)
def P_Da(F0, K, r, sigma, T, beta):
    return P_Ba(F0/beta, K + ((1 - beta)/beta) * F0, r, beta * sigma, T)

In [28]:
# Validate each of the formulae above using the following values:
S0_1 = 80   # initial stock price < strike price, i.e., S0 < K
S0_2 = 90   # initial stock price > strike price, i.e., S0 > K
K = 85
r = 0.05
sigma = 0.40
T = 0.2
F0_1, F0_2 = S0_1 * np.exp(r * T), S0_2 * np.exp(r * T)
beta = 0.3
# Implementation
print("Black-Scholes model")
print("Vanilla")
print("Call")
print(f"C_BSv (S0 = {S0_1}) =", "{:.3f}".format(C_BSv(S0_1, K, r, sigma, T)))
print(f"C_BSv (S0 = {S0_2}) =", "{:.3f}".format(C_BSv(S0_2, K, r, sigma, T)))
print("Put")
print(f"P_BSv (S0 = {S0_1}) =", "{:.3f}".format(P_BSv(S0_1, K, r, sigma, T)))
print(f"P_BSv (S0 = {S0_2}) =", "{:.3f}".format(P_BSv(S0_2, K, r, sigma, T)))
print("")
print("Cash-or-nothing")
print("Call")
print(f"C_BSc (S0 = {S0_1}) =", "{:.3f}".format(C_BSc(S0_1, K, r, sigma, T)))
print(f"C_BSc (S0 = {S0_2}) =", "{:.3f}".format(C_BSc(S0_2, K, r, sigma, T)))
print("Put")
print(f"P_BSc (S0 = {S0_1}) =", "{:.3f}".format(P_BSc(S0_1, K, r, sigma, T)))
print(f"P_BSc (S0 = {S0_2}) =", "{:.3f}".format(P_BSc(S0_2, K, r, sigma, T)))
print("")
print("Asset-or-nothing")
print("Call")
print(f"C_BSa (S0 = {S0_1}) =", "{:.3f}".format(C_BSa(S0_1, K, r, sigma, T)))
print(f"C_BSa (S0 = {S0_2}) =", "{:.3f}".format(C_BSa(S0_2, K, r, sigma, T)))
print("Put")
print(f"P_BSa (S0 = {S0_1}) =", "{:.3f}".format(P_BSa(S0_1, K, r, sigma, T)))
print(f"P_BSa (S0 = {S0_2}) =", "{:.3f}".format(P_BSa(S0_2, K, r, sigma, T)))
print("")
print("Bachelier model")
print("Vanilla")
print("Call")
print(f"C_Bav (S0 = {S0_1}) =", "{:.3f}".format(C_Bav(S0_1, K, r, sigma, T)))
print(f"C_Bav (S0 = {S0_2}) =", "{:.3f}".format(C_Bav(S0_2, K, r, sigma, T)))
print("Put")
print(f"P_Bav (S0 = {S0_1}) =", "{:.3f}".format(P_Bav(S0_1, K, r, sigma, T)))
print(f"P_Bav (S0 = {S0_2}) =", "{:.3f}".format(P_Bav(S0_2, K, r, sigma, T)))
print("")
print("Cash-or-nothing")
print("Call")
print(f"C_Bac (S0 = {S0_1}) =", "{:.3f}".format(C_Bac(S0_1, K, r, sigma, T)))
print(f"C_Bac (S0 = {S0_2}) =", "{:.3f}".format(C_Bac(S0_2, K, r, sigma, T)))
print("Put")
print(f"P_Bac (S0 = {S0_1}) =", "{:.3f}".format(P_Bac(S0_1, K, r, sigma, T)))
print(f"P_Bac (S0 = {S0_2}) =", "{:.3f}".format(P_Bac(S0_2, K, r, sigma, T)))
print("")
print("Asset-or-nothing")
print("Call")
print(f"C_Baa (S0 = {S0_1}) =", "{:.3f}".format(C_Baa(S0_1, K, r, sigma, T)))
print(f"C_Baa (S0 = {S0_2}) =", "{:.3f}".format(C_Baa(S0_2, K, r, sigma, T)))
print("Put")
print(f"P_Baa (S0 = {S0_1}) =", "{:.3f}".format(P_Baa(S0_1, K, r, sigma, T)))
print(f"P_Baa (S0 = {S0_2}) =", "{:.3f}".format(P_Baa(S0_2, K, r, sigma, T)))
print("")
print("Black model")
print("Vanilla")
print("Call")
print(f"C_Bv (F0 = {F0_1}) =", "{:.3f}".format(C_Bv(F0_1, K, r, sigma, T)))
print(f"C_Bv (F0 = {F0_2}) =", "{:.3f}".format(C_Bv(F0_2, K, r, sigma, T)))
print("Put")
print(f"P_Bv (F0 = {F0_1}) =", "{:.3f}".format(P_Bv(F0_1, K, r, sigma, T)))
print(f"P_Bv (F0 = {F0_2}) =", "{:.3f}".format(P_Bv(F0_2, K, r, sigma, T)))
print("")
print("Cash-or-nothing")
print("Call")
print(f"C_Bc (F0 = {F0_1}) =", "{:.3f}".format(C_Bc(F0_1, K, r, sigma, T)))
print(f"C_Bc (F0 = {F0_2}) =", "{:.3f}".format(C_Bc(F0_2, K, r, sigma, T)))
print("Put")
print(f"P_Bc (F0 = {F0_1}) =", "{:.3f}".format(P_Bc(F0_1, K, r, sigma, T)))
print(f"P_Bc (F0 = {F0_2}) =", "{:.3f}".format(P_Bc(F0_2, K, r, sigma, T)))
print("")
print("Asset-or-nothing")
print("Call")
print(f"C_Ba (F0 = {F0_1}) =", "{:.3f}".format(C_Ba(F0_1, K, r, sigma, T)))
print(f"C_Ba (F0 = {F0_2}) =", "{:.3f}".format(C_Ba(F0_2, K, r, sigma, T)))
print("Put")
print(f"P_Ba (F0 = {F0_1}) =", "{:.3f}".format(P_Ba(F0_1, K, r, sigma, T)))
print(f"P_Ba (F0 = {F0_2}) =", "{:.3f}".format(P_Ba(F0_2, K, r, sigma, T)))
print("")
print("Displaced-diffusion model")
print("Vanilla")
print("Call")
print(f"C_Dv (F0 = {F0_1}) =", "{:.3f}".format(C_Dv(F0_1, K, r, sigma, T, beta)))
print(f"C_Dv (F0 = {F0_2}) =", "{:.3f}".format(C_Dv(F0_2, K, r, sigma, T, beta)))
print("Put")
print(f"P_Dv (F0 = {F0_1}) =", "{:.3f}".format(P_Dv(F0_1, K, r, sigma, T, beta)))
print(f"P_Dv (F0 = {F0_2}) =", "{:.3f}".format(P_Dv(F0_2, K, r, sigma, T, beta)))
print("")
print("Cash-or-nothing")
print("Call")
print(f"C_Dc (F0 = {F0_1}) =", "{:.3f}".format(C_Dc(F0_1, K, r, sigma, T, beta)))
print(f"C_Dc (F0 = {F0_2}) =", "{:.3f}".format(C_Dc(F0_2, K, r, sigma, T, beta)))
print("Put")
print(f"P_Dc (F0 = {F0_1}) =", "{:.3f}".format(P_Dc(F0_1, K, r, sigma, T, beta)))
print(f"P_Dc (F0 = {F0_2}) =", "{:.3f}".format(P_Dc(F0_2, K, r, sigma, T, beta)))
print("")
print("Asset-or-nothing")
print("Call")
print(f"C_Da (F0 = {F0_1}) =", "{:.3f}".format(C_Da(F0_1, K, r, sigma, T, beta)))
print(f"C_Da (F0 = {F0_2}) =", "{:.3f}".format(C_Da(F0_2, K, r, sigma, T, beta)))
print("Put")
print(f"P_Da (F0 = {F0_1}) =", "{:.3f}".format(P_Da(F0_1, K, r, sigma, T, beta)))
print(f"P_Da (F0 = {F0_2}) =", "{:.3f}".format(P_Da(F0_2, K, r, sigma, T, beta)))

Black-Scholes model
Vanilla
Call
C_BSv (S0 = 80) = 4.005
C_BSv (S0 = 90) = 9.560
Put
P_BSv (S0 = 80) = 8.159
P_BSv (S0 = 90) = 3.714

Cash-or-nothing
Call
C_BSc (S0 = 80) = 0.351
C_BSc (S0 = 90) = 0.606
Put
P_BSc (S0 = 80) = 0.639
P_BSc (S0 = 90) = 0.384

Asset-or-nothing
Call
C_BSa (S0 = 80) = 33.861
C_BSa (S0 = 90) = 61.109
Put
P_BSa (S0 = 80) = 46.139
P_BSa (S0 = 90) = 28.891

Bachelier model
Vanilla
Call
C_Bav (S0 = 80) = 0.000
C_Bav (S0 = 90) = 4.950
Put
P_Bav (S0 = 80) = 4.950
P_Bav (S0 = 90) = 0.000

Cash-or-nothing
Call
C_Bac (S0 = 80) = 0.000
C_Bac (S0 = 90) = 0.990
Put
P_Bac (S0 = 80) = 0.990
P_Bac (S0 = 90) = 0.000

Asset-or-nothing
Call
C_Baa (S0 = 80) = 0.000
C_Baa (S0 = 90) = 90.000
Put
P_Baa (S0 = 80) = 80.000
P_Baa (S0 = 90) = 0.000

Black model
Vanilla
Call
C_Bv (F0 = 80.80401336673344) = 4.005
C_Bv (F0 = 90.90451503757511) = 9.560
Put
P_Bv (F0 = 80.80401336673344) = 8.159
P_Bv (F0 = 90.90451503757511) = 3.714

Cash-or-nothing
Call
C_Bc (F0 = 80.80401336673344) = 0.351